Install Dependencies

In [0]:
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 70.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Configure Kaggle Credentials

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "siddiquiaddy"
os.environ["KAGGLE_KEY"] = "KGAT_c3901508c4177a50054a13f740d46748"

print("Kaggle credentials configured!")



Kaggle credentials configured!


Create Database Schema

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

Create Volume for Data Storage

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

Download Dataset from Kaggle

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:25<00:00, 179MB/s]


Extract Downloaded Dataset

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-90cb8a35-e52a-4017-8548-23 nogroup 8.4G Jan  9 04:48 2019-Nov.csv
-rwxrwxrwx 1 spark-90cb8a35-e52a-4017-8548-23 nogroup 5.3G Jan  9 04:50 2019-Oct.csv
-rwxrwxrwx 1 spark-90cb8a35-e52a-4017-8548-23 nogroup 4.3G Jan  9 04:46 ecommerce-behavior-data-from-multi-category-store.zip


Clean Up Zip File

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 spark-90cb8a35-e52a-4017-8548-23 nogroup 8.4G Jan  9 04:48 2019-Nov.csv
-rwxrwxrwx 1 spark-90cb8a35-e52a-4017-8548-23 nogroup 5.3G Jan  9 04:50 2019-Oct.csv


Restart Python Environment

In [0]:
%restart_python

Load November 2019 Data

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

Load October 2019 Data

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

Display Dataset Statistics and Schema

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

October 2019 - Total Events: 42,448,765

SCHEMA:
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



Display Sample Data

In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+------+---------+------------------------------------+
|_c0                    |_c1       |_c2       |_c3                |_c4                                |_c5     |_c6   |_c7      |_c8                                 |
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                      |brand   |price |user_id  |user_session                        |
|2019-10-01 00:00:00 UTC|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79 |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.20 |554748717|9333dfb